In [0]:

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.c1 = nn.Conv2d(1, 8, 3, padding=1) #RF #input -? OUtput? RF
        #self.c2 = nn.Conv2d(8, 16, 3, padding=1) #RF3
        #self.c3 = nn.Conv2d(16, 32, 3, padding=1) #RF 5
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.c4 = nn.Conv2d(32, 64, 3, padding=1) #RF 10      
        #self.c5 = nn.Conv2d(64, 128, 3, padding=1) #RF 12
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.c6 = nn.Conv2d(128, 256, 3, padding=1) #RF 24
        #self.c7 = nn.Conv2d(256, 512, 3, padding=1) #RF 26
        #self.c8= nn.Conv2d(512, 10, 3) #RF 28
        #self.avg_pool = nn.AvgPool2d(kernel_size=3)
        
        self.c1 = nn.Conv2d(1, 8, 3, padding=1) #RF #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(8)
        self.dropout1 = nn.Dropout(0.15)
        self.c2 = nn.Conv2d(8, 16, 3, padding=1) #RF3
        self.bn2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.15)
        self.c3 = nn.Conv2d(16, 16, 3, padding=1) #RF 5
        self.bn3 = nn.BatchNorm2d(16)
        self.dropout3 = nn.Dropout(0.15)

        self.pool1 = nn.MaxPool2d(2, 2)
        

        self.c4 = nn.Conv2d(16, 16, 3, padding=1) #RF 10 
        self.bn4 = nn.BatchNorm2d(16)     
        self.dropout4 = nn.Dropout(0.15)
        self.c5 = nn.Conv2d(16, 16, 3, padding=1) #RF 12
        self.bn5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout(0.15)

        self.pool2 = nn.MaxPool2d(2, 2)
        

        self.c6 = nn.Conv2d(16, 16, 3, padding=1) #RF 24
        self.bn6 = nn.BatchNorm2d(16)
        self.dropout6 = nn.Dropout(0.15)

        self.c7 = nn.Conv2d(16, 32, 3, padding=1) #RF 26
        self.bn7 = nn.BatchNorm2d(32)
        self.dropout7 = nn.Dropout(0.15)

        self.c8= nn.Conv2d(32, 10, 3) #RF 28
        self.avg_pool = nn.AvgPool2d(kernel_size=3)
     
              

    def forward(self, x):
        #x = self.pool1(F.relu(self.c3(F.relu(self.c2(self.c1(x))))))
        #x = self.pool2(F.relu(self.c5(F.relu(self.c4(x)))))
        #x = F.relu(self.c7(F.relu(self.c6(x))))
        #x = F.relu(self.c8(x))
        #x = self.avg_pool(x)
        #x = x.view(-1, 10)
        #return F.log_softmax(x)
        #x = self.pool1(self.bn3(F.relu(self.c3(self.bn2(F.relu(self.c2(self.bn1(F.relu(self.c1(x))))))))))
        #x = self.pool2(self.bn5(F.relu(self.c5(self.bn4(F.relu(self.c4(x)))))))
        #x = self.bn7(F.relu(self.c7(self.bn6(F.relu(self.c6(x))))))
        #x = F.relu(self.c8(x))
        #x = self.avg_pool(x)
        #x = x.view(-1, 10)
        #return F.log_softmax(x)
        x = self.pool1(self.dropout3(self.bn3(F.relu(self.c3(self.dropout2(self.bn2(F.relu(self.c2(self.dropout1(self.bn1(F.relu(self.c1(x)))))))))))))
        x = self.pool2(self.dropout5(self.bn5(F.relu(self.c5(self.dropout4(self.bn4(F.relu(self.c4(x)))))))))
        x = self.dropout7(self.bn7(F.relu(self.c7(self.dropout6(self.bn6(F.relu(self.c6(x))))))))
        x = F.relu(self.c8(x))
        x = self.avg_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [20]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]           2,320
       BatchNorm2d-8           [-1, 16, 28, 28]              32
           Dropout-9           [-1, 16, 28, 28]               0
        MaxPool2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0862, Accuracy: 9745/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0737, Accuracy: 9769/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0253, Accuracy: 9913/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0263, Accuracy: 9914/10000 (99%)



loss=0.011858056299388409 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.60it/s]
